In [2]:
import pandas as pd
import torch
from src.GP.data import *

/Users/jake/Python/Obs_and_exp_data/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ihdp_table = pd.read_csv('src/data/ihdp.csv')
ihdp_full_table = pd.read_csv('src/data/ihdpFull.csv')

In [4]:
ihdp_table

,treat,bw,b.head,preterm,birth.o,nnhealth,momage,sex,twin,b.marr,...,ark,ein,har,mia,pen,tex,was,momwhite,momblack,momhisp
0,1,1559,28.648521,10,2,94,33,1,0,1,...,1,0,0,0,0,0,0,1,0,0
1,1,1420,27.000000,4,2,85,15,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,1000,25.000000,8,4,89,33,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,0,1430,29.000000,6,1,112,22,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,1984,31.000000,2,1,99,20,0,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,1,2140,29.000000,5,1,112,32,0,0,1,...,0,0,0,0,0,0,0,1,0,0
981,0,2350,30.000000,3,2,111,28,0,0,0,...,0,0,0,0,0,0,0,0,1,0
982,0,1670,29.000000,3,1,125,28,1,0,1,...,0,0,0,0,0,0,0,1,0,0
983,0,1740,31.000000,6,1,107,26,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [16]:
n_samples_exp = 100
T_prop = 0.5
W_prop = 0.4
WT_prop = 0.3
sigma_noise = 0.5

n_samples_obs = 1000

In [17]:
df_obs = ihdp_table.sample(n=n_samples_obs,replace=True)
T_obs = torch.tensor(df_obs.treat.values)
X_obs = torch.tensor(df_obs.iloc[:,1:].values).type(torch.float)
d = X_obs.shape[1]

In [18]:
w0 = (torch.randn(d)* (torch.rand(d) >(1-W_prop))).type(torch.float)
w1 = (torch.randn(d)* (torch.rand(d) >(1-W_prop))).type(torch.float)
cfd_GPs = [0,0]
ucfd_GPs = [0,0]

for i in range(2):
    cfd_GPs[i] =GP_func((-1,1),kernel="RFF",num_samples_RFF=1500,d=d)

    ucfd_GPs[i] =GP_func((-1,1),train_data=None,scale=1,kernel="RFF",num_samples_RFF=1500,d=d)

In [19]:
cfded_GP_func = lambda X,T: (X @ w0) + T*(X_obs @ w1)
ucfded_GP_func = lambda X,T: cfded_GP_func(X,T) + (1-T)*ucfd_GPs[0](X) + (T)*ucfd_GPs[1](X)
outcome_funcs_GP = outcome_funcs(cfounded_func=cfded_GP_func,uncfounded_func=ucfded_GP_func)

In [20]:
df_obs = ihdp_table.sample(n=n_samples_obs,replace=True)
T_obs = torch.tensor(df_obs.treat.values)
X_obs = torch.tensor(df_obs.iloc[:,1:].values).type(torch.float)
d = X_obs.shape[1]

w0 = (torch.randn(d)* (torch.rand(d) >(1-W_prop))).type(torch.float)
w1 = (torch.randn(d)* (torch.rand(d) >(1-W_prop))).type(torch.float)

Y_obs = cfded_GP_func(X_obs,T_obs) + sigma_noise*(torch.randn(len(T_obs)))

In [22]:
ucfded_GP_func(X_exp,T_exp)

RuntimeError: The size of tensor a (100) must match the size of tensor b (1000) at non-singleton dimension 0

In [21]:
df_exp = ihdp_table.sample(n=n_samples_exp,weights=((0.8*(ihdp_table.cig))* (0.8*(ihdp_table.sex))),replace=True)
T_exp = (torch.rand(n_samples_exp) > (1-T_prop)).type(torch.float)
X_exp = torch.tensor(df_exp.iloc[:,1:].values).type(torch.float)


w0 = (torch.randn(28)* (torch.rand(28) >(1-W_prop))).type(torch.float)
w1 = (torch.randn(28)* (torch.rand(28) >(1-W_prop))).type(torch.float)

Y_exp = ucfded_GP_func(X_exp,T_exp) + sigma_noise*(torch.randn(len(T_exp)))

obs_data_GP = data(X=X_obs,Y=Y_obs,T=T_obs)
exp_data_GP = data(X=X_exp,Y=Y_exp,T=T_exp)

RuntimeError: The size of tensor a (100) must match the size of tensor b (1000) at non-singleton dimension 0

In [ ]:
get_train_data_IHDP_Linear()

/var/folders/v1/x4lr48v53397r9xsq4q70mvw0000gn/T/ipykernel_3633/4083587808.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     -0.518060
1     -0.822071
2     -1.740668
3     -0.800200
4      0.411472
         ...   
980    0.752665
981    1.211963
982   -0.275288
983   -0.122189
984    0.446466
Name: bw, Length: 985, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ihdp_table.iloc[:,1:7] = (ihdp_table.iloc[:,1:7] - ihdp_table.iloc[:,1:7].mean())/(ihdp_table.iloc[:,1:7].std())
/var/folders/v1/x4lr48v53397r9xsq4q70mvw0000gn/T/ipykernel_3633/4083587808.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      1.126525
1     -1.112884
2      0.380055
3     -0.366414
4     -1.859354
         ...   
980   -0.739649
981   -1.486119
982   -1.486119
983   -0.366414
984   -0.739649
Name: p

(data(X=tensor([[ 0.4137, -0.1734, -0.7396,  ...,  1.0000,  0.0000,  0.0000],
         [-1.5876, -1.3898,  1.1265,  ...,  1.0000,  0.0000,  0.0000],
         [ 1.5182,  1.0430, -0.3664,  ...,  1.0000,  0.0000,  0.0000],
         ...,
         [-0.0128, -0.5789, -0.3664,  ...,  1.0000,  0.0000,  0.0000],
         [-1.5876, -1.3898,  1.1265,  ...,  1.0000,  0.0000,  0.0000],
         [-0.9533, -0.1734,  0.0068,  ...,  1.0000,  0.0000,  0.0000]]), T=tensor([0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1.,
         1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
         0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
         0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
         1., 1., 0., 1., 0., 1., 1., 0., 0., 1.]), Y=tensor([ 1.4588e+00, -1.8454e+00,  1.1625e+00, -5.7932e-01,  1.2991e+00,
         -3.7292e+0

In [ ]:
ihdp_table["U"] = torch.randn((len(ihdp_table),3))

ValueError: Expected a 1D array, got an array with shape (985, 3)

In [ ]:
ihdp_table[1:7]

,treat,bw,b.head,preterm,birth.o,nnhealth,momage,sex,twin,b.marr,...,ark,ein,har,mia,pen,tex,was,momwhite,momblack,momhisp
1,1,1420,27.0,4,2,85,15,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,1000,25.0,8,4,89,33,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,0,1430,29.0,6,1,112,22,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,1984,31.0,2,1,99,20,0,0,0,...,1,0,0,0,0,0,0,0,1,0
5,0,1320,28.0,7,2,110,23,1,0,0,...,1,0,0,0,0,0,0,0,1,0
6,1,2240,31.0,3,2,105,22,1,0,0,...,1,0,0,0,0,0,0,0,1,0


In [ ]:
def gen_hidden_cov(df, n_hidden_cov = 3):
    
    return

SyntaxError: expected default value expression (1152049222.py, line 1)

In [ ]:
ihdp_full_table

,sitenum,momrace,bmarr,workdur,cccats3,treat,prenatal,first,bw,preterm,...,other60,mom248,relative248,nanny248,familydc248,center248,other248,ccurr60,nvisitt,ncdct
0,1,3,1,1,3,1,1,1,1559,10,...,0,0,0,0,0,1,0,1,72,432
1,1,1,0,1,1,1,0,1,1420,4,...,0,0,1,0,0,0,0,1,60,391
2,1,1,0,1,2,0,1,1,1000,8,...,0,0,1,0,0,0,0,1,0,0
3,1,1,0,0,1,0,1,2,1430,6,...,0,0,0,0,0,1,0,1,0,0
4,1,1,0,0,2,0,1,2,1984,2,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,8,3,1,1,2,1,1,2,2140,5,...,0,0,0,0,0,1,0,1,56,188
981,8,1,0,1,2,0,1,1,2350,3,...,0,0,1,0,0,0,0,1,0,0
982,8,3,1,1,3,0,1,2,1670,3,...,0,0,1,0,0,0,0,1,0,0
983,8,3,1,1,1,0,1,2,1740,6,...,0,0,1,0,0,0,0,1,0,0
